In [1]:
import readfof
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
        .master("spark://master:7077")\
        .appName("CosmoSparkApplication")\
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/28 15:50:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
sc = spark.sparkContext

In [4]:
# Read data
def read_cosmo_data(file_path):

    # Read Fof
    FoF = readfof.FoF_catalog(
        file_path,           # simulation directory
        2,                  # snapnum, indicating the redshift (z=1)
        long_ids = False,
        swap = False,
        SFR = False,
        read_IDs = False
        )
    
    # Get positions and masses
    # pos = FoF.GroupPos/1e06             # Halo positions in Gpc/h 
    # mass_raw = FoF.GroupMass * 1e10     # Halo masses in Msun/h

    # return (pos, mass_raw)

    return FoF

In [35]:
# get masses and positions

def get_pos_mass(FoF):

    pos = FoF.GroupPos/1e06             # Halo positions in Gpc/h 
    mass_raw = FoF.GroupMass * 1e10     # Halo masses in Msun/h

    return pos, mass_raw

In [5]:
# path list
path_list = ["/mnt/cosmo_GNN/Data/"+str(i) for i in range(50)]

In [20]:
# RDD
cosmo_rdd = sc.parallelize(path_list)\
            .map(read_cosmo_data)

In [21]:
cosmo_rdd.count()

50

In [22]:
cosmo_rdd.getNumPartitions()

16

In [36]:
pos_mass_rdd = cosmo_rdd.map(get_pos_mass)

In [44]:
cosa = pos_mass_rdd.take(2)

In [45]:
len(cosa)

2

In [46]:
type(cosa)

list

In [48]:
cosa[1][1]

array([1.13813613e+15, 1.03374206e+15, 9.20060714e+14, ...,
       8.84651170e+12, 8.84651170e+12, 8.84651170e+12], dtype=float32)

In [ ]:
def mass_filter(pos_mass_list):

    mass = pos_mass_list[0][1]
    pos = pos_mass_list[0][0]
    cut = np.quantile(mass, 0.997)
    mask = (pos_mass_list >= cut)
    mass_filtered = mass[mask]
    pos_filtered = pos[mask]

    return pos_filtered, mass_filtered


In [ ]:
filtered_rdd = cosmo_rdd.map()

In [14]:
sc.stop()
spark.stop()